# Step 5: Network Optimization

**Iteratively optimize 20 networks to match reference distributions**

This notebook:
1. Loads 20 initial networks from Step 4 (with syntax metrics)
2. Defines optimization objectives based on reference city distributions
3. Iteratively optimizes each network:
   - Match segment length distribution
   - Match orientation distribution
   - Match degree distribution
   - Improve intelligibility
   - Adjust node density
4. Uses simulated annealing / genetic algorithm for optimization
5. Saves optimized networks for ranking and selection

**Optimization strategies:**
- Add/remove edges to match degree distribution
- Perturb node positions to match segment lengths
- Remove/add edges to match orientation histogram
- Multiple iterations with cooling schedule

In [ ]:
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from pathlib import Path
import pickle
from scipy import stats
from scipy.spatial import Delaunay
import copy
import math

%matplotlib inline
plt.rcParams['figure.dpi'] = 100
plt.rcParams['font.size'] = 10

print("✓ Libraries loaded")

## Configuration

In [ ]:
WINDOW_SIZE_M = 500  # 500×500m window

# Optimization parameters
NUM_ITERATIONS = 100  # Number of optimization iterations per network
INITIAL_TEMP = 1.0    # Initial temperature for simulated annealing
COOLING_RATE = 0.95   # Temperature cooling rate
MIN_TEMP = 0.01       # Minimum temperature

# Create output directories
Path("outputs/generated/visualizations").mkdir(parents=True, exist_ok=True)
Path("outputs/generated/optimized").mkdir(parents=True, exist_ok=True)

print(f"Window size: {WINDOW_SIZE_M}m × {WINDOW_SIZE_M}m")
print(f"Optimization iterations: {NUM_ITERATIONS}")
print(f"Temperature: {INITIAL_TEMP} → {MIN_TEMP} (cooling: {COOLING_RATE})")
print("✓ Configuration set")

## Load Data

In [ ]:
# Load reference city data
with open('outputs/data/reference_cities_data.pkl', 'rb') as f:
    reference_data = pickle.load(f)

# Load generated networks with space syntax from Step 4
with open('outputs/generated/syntax/networks_with_syntax_20.pkl', 'rb') as f:
    generated_networks = pickle.load(f)

print("✓ Loaded reference data from Step 1")
print(f"✓ Loaded {len(generated_networks)} networks from Step 4")

# Reference city for optimization (London)
reference_city = 'london'
ref_data = reference_data[reference_city]

print(f"\nUsing {reference_city.upper()} as optimization target")

## Extract Reference Distributions

In [ ]:
# Extract target distributions from reference city
ref_segment_lengths = ref_data['morphology']['segment_lengths']
ref_degree_dist = ref_data['morphology']['degree_distribution']
ref_orientation_hist = ref_data['morphology']['orientation_hist']

# Target metrics (adjusted for doubled roads where needed)
target_metrics = {
    'node_density': ref_data['morphology']['node_density'] / 2,  # HALVED
    'avg_degree': ref_data['morphology']['avg_degree'] / 2,      # HALVED
    'avg_segment_length': ref_data['morphology']['avg_segment_length'],  # NOT doubled
    'intelligibility': ref_data['syntax']['intelligibility'],
    'mean_depth': ref_data['syntax']['mean_depth']
}

# Compute segment length probability distribution
seg_counts, seg_bins = np.histogram(ref_segment_lengths, bins=30, range=(5, 150))
seg_probs = seg_counts / seg_counts.sum()

# Orientation probability distribution
ori_bins, ori_counts = ref_orientation_hist
ori_probs = ori_counts / ori_counts.sum() if ori_counts.sum() > 0 else ori_counts

print("Target Metrics:")
print("="*50)
for k, v in target_metrics.items():
    print(f"  {k:25s}: {v:.3f}")
print("="*50)

print(f"\nSegment length distribution: {len(seg_bins)-1} bins")
print(f"Orientation distribution: {len(ori_bins)-1} bins")
print(f"Degree distribution: {len(ref_degree_dist)} unique degrees")

## Optimization Functions

In [ ]:
def compute_fitness(G, pos, target_metrics, seg_probs, seg_bins, ori_probs, ori_bins):
    """
    Compute fitness score for a network.
    Lower score = better fit to target distributions.
    
    Components:
    1. Segment length distribution similarity (KL divergence)
    2. Orientation distribution similarity (KL divergence)
    3. Metric differences (node density, avg degree, intelligibility)
    """
    fitness = 0.0
    
    # 1. Segment length distribution
    lengths = [d['length'] for u, v, d in G.edges(data=True) if 'length' in d]
    if lengths:
        counts, _ = np.histogram(lengths, bins=seg_bins)
        probs = counts / counts.sum() if counts.sum() > 0 else counts
        probs = np.clip(probs, 1e-10, 1)  # Avoid log(0)
        seg_probs_clipped = np.clip(seg_probs, 1e-10, 1)
        kl_seg = np.sum(probs * np.log(probs / seg_probs_clipped))
        fitness += kl_seg * 2.0  # Weight
    
    # 2. Orientation distribution
    bearings = []
    for u, v in G.edges():
        dx = pos[v][0] - pos[u][0]
        dy = pos[v][1] - pos[u][1]
        angle = math.atan2(dy, dx)
        bearing = math.degrees(angle) % 180
        bearings.append(bearing)
    
    if bearings:
        counts, _ = np.histogram(bearings, bins=ori_bins)
        probs = counts / counts.sum() if counts.sum() > 0 else counts
        probs = np.clip(probs, 1e-10, 1)
        ori_probs_clipped = np.clip(ori_probs, 1e-10, 1)
        kl_ori = np.sum(probs * np.log(probs / ori_probs_clipped))
        fitness += kl_ori * 1.5  # Weight
    
    # 3. Metric differences
    area_km2 = (WINDOW_SIZE_M / 1000.0) ** 2
    node_density = G.number_of_nodes() / area_km2
    degrees = [d for _, d in G.degree()]
    avg_degree = np.mean(degrees) if degrees else 0
    avg_seg_len = np.mean(lengths) if lengths else 0
    
    # Normalized differences
    fitness += abs(node_density - target_metrics['node_density']) / target_metrics['node_density'] * 1.0
    fitness += abs(avg_degree - target_metrics['avg_degree']) / target_metrics['avg_degree'] * 1.0
    fitness += abs(avg_seg_len - target_metrics['avg_segment_length']) / target_metrics['avg_segment_length'] * 1.5
    
    return fitness


def perturb_network(G, pos, temperature):
    """
    Apply random perturbation to network.
    
    Operations:
    1. Move random node (small displacement)
    2. Add random edge (with probability)
    3. Remove random edge (with probability)
    """
    G_new = G.copy()
    pos_new = pos.copy()
    
    # Choose operation based on temperature
    operation = np.random.choice(['move_node', 'add_edge', 'remove_edge'], p=[0.5, 0.25, 0.25])
    
    if operation == 'move_node' and len(pos_new) > 0:
        # Move random node
        node = np.random.choice(list(pos_new.keys()))
        max_displacement = temperature * 20  # Scale with temperature
        dx = np.random.uniform(-max_displacement, max_displacement)
        dy = np.random.uniform(-max_displacement, max_displacement)
        
        new_x = np.clip(pos_new[node][0] + dx, 10, WINDOW_SIZE_M - 10)
        new_y = np.clip(pos_new[node][1] + dy, 10, WINDOW_SIZE_M - 10)
        pos_new[node] = (new_x, new_y)
        
        # Update edge lengths for ALL edges connected to this node
        # For undirected graphs, neighbors() gives all connected nodes
        for neighbor in list(G_new.neighbors(node)):
            length = np.sqrt((pos_new[node][0] - pos_new[neighbor][0])**2 + 
                           (pos_new[node][1] - pos_new[neighbor][1])**2)
            if G_new.has_edge(node, neighbor):
                G_new[node][neighbor]['length'] = length
    
    elif operation == 'add_edge' and len(pos_new) > 1:
        # Add random edge between nearby nodes
        nodes = list(G_new.nodes())
        u = np.random.choice(nodes)
        
        # Find nearby nodes
        candidates = []
        for v in nodes:
            if v != u and not G_new.has_edge(u, v):
                dist = np.sqrt((pos_new[u][0] - pos_new[v][0])**2 + 
                             (pos_new[u][1] - pos_new[v][1])**2)
                if dist < 100:  # Max connection distance
                    candidates.append((v, dist))
        
        if candidates:
            v, length = candidates[np.random.randint(len(candidates))]
            G_new.add_edge(u, v, length=length)
    
    elif operation == 'remove_edge' and G_new.number_of_edges() > 0:
        # Remove random edge (but don't disconnect graph)
        edges = list(G_new.edges())
        if edges:
            u, v = edges[np.random.randint(len(edges))]
            G_new.remove_edge(u, v)
            
            # Check if still connected
            if not nx.is_connected(G_new):
                # Restore edge if it disconnects graph
                length = np.sqrt((pos_new[u][0] - pos_new[v][0])**2 + 
                               (pos_new[u][1] - pos_new[v][1])**2)
                G_new.add_edge(u, v, length=length)
    
    return G_new, pos_new


def optimize_network(G_init, pos_init, target_metrics, seg_probs, seg_bins, ori_probs, ori_bins,
                    num_iterations=100, initial_temp=1.0, cooling_rate=0.95):
    """
    Optimize network using simulated annealing.
    
    HOW IT WORKS:
    1. Start with initial network and compute fitness (how far from target)
    2. Each iteration:
       - Randomly perturb network (move node, add/remove edge)
       - Compute new fitness
       - Accept if better, or accept worse with probability exp(-delta/T)
       - Cool down temperature
    3. Return best network found
    
    Fitness measures:
    - KL divergence between segment length distributions
    - KL divergence between orientation distributions  
    - Difference in node density, avg degree, avg segment length
    
    Returns:
        Optimized graph, positions, fitness history
    """
    G_current = G_init.copy()
    pos_current = pos_init.copy()
    fitness_current = compute_fitness(G_current, pos_current, target_metrics, 
                                     seg_probs, seg_bins, ori_probs, ori_bins)
    
    G_best = G_current.copy()
    pos_best = pos_current.copy()
    fitness_best = fitness_current
    
    fitness_history = [fitness_current]
    temperature = initial_temp
    
    for iteration in range(num_iterations):
        # Perturb network
        G_new, pos_new = perturb_network(G_current, pos_current, temperature)
        
        # Compute new fitness
        fitness_new = compute_fitness(G_new, pos_new, target_metrics,
                                     seg_probs, seg_bins, ori_probs, ori_bins)
        
        # Accept or reject (simulated annealing)
        delta = fitness_new - fitness_current
        
        if delta < 0 or np.random.random() < np.exp(-delta / temperature):
            # Accept
            G_current = G_new
            pos_current = pos_new
            fitness_current = fitness_new
            
            # Update best
            if fitness_current < fitness_best:
                G_best = G_current.copy()
                pos_best = pos_current.copy()
                fitness_best = fitness_current
        
        fitness_history.append(fitness_current)
        
        # Cool down
        temperature = max(temperature * cooling_rate, MIN_TEMP)
        
        # Progress
        if (iteration + 1) % 20 == 0:
            print(f"    Iter {iteration+1:3d}: fitness={fitness_current:.4f}, best={fitness_best:.4f}, T={temperature:.3f}")
    
    return G_best, pos_best, fitness_history


print("✓ Optimization functions defined")

## Optimize All 20 Networks

In [ ]:
print("Optimizing 20 networks...")
print("="*70)

optimized_networks = []

for idx, network_data in enumerate(generated_networks):
    net_id = network_data['id']
    G_init = network_data['graph']
    pos_init = network_data['pos']
    
    print(f"\nNetwork {net_id+1:2d}/{len(generated_networks)}:")
    print(f"  Initial: {G_init.number_of_nodes()} nodes, {G_init.number_of_edges()} edges")
    
    # Compute initial fitness
    fitness_init = compute_fitness(G_init, pos_init, target_metrics,
                                   seg_probs, seg_bins, ori_probs, ori_bins)
    print(f"  Initial fitness: {fitness_init:.4f}")
    
    # Optimize
    G_opt, pos_opt, fitness_history = optimize_network(
        G_init, pos_init, target_metrics, seg_probs, seg_bins, ori_probs, ori_bins,
        num_iterations=NUM_ITERATIONS,
        initial_temp=INITIAL_TEMP,
        cooling_rate=COOLING_RATE
    )
    
    print(f"  Final: {G_opt.number_of_nodes()} nodes, {G_opt.number_of_edges()} edges")
    print(f"  Final fitness: {fitness_history[-1]:.4f}")
    print(f"  Improvement: {(fitness_init - fitness_history[-1]) / fitness_init * 100:.1f}%")
    
    # Store optimized network
    optimized_data = {
        'id': net_id,
        'graph': G_opt,
        'pos': pos_opt,
        'fitness_history': fitness_history,
        'initial_fitness': fitness_init,
        'final_fitness': fitness_history[-1],
        'improvement': (fitness_init - fitness_history[-1]) / fitness_init
    }
    optimized_networks.append(optimized_data)

print("\n" + "="*70)
print("✓ Optimization complete for all 20 networks")

## Optimization Results Summary

In [ ]:
print("\n" + "="*70)
print(" "*20 + "OPTIMIZATION SUMMARY")
print("="*70)
print(f"{'Network':<10} {'Initial':<12} {'Final':<12} {'Improvement':<15}")
print("-"*70)

for net in optimized_networks:
    net_id = net['id'] + 1
    init_fit = net['initial_fitness']
    final_fit = net['final_fitness']
    improvement = net['improvement'] * 100
    
    print(f"{net_id:<10} {init_fit:<12.4f} {final_fit:<12.4f} {improvement:<15.1f}%")

print("="*70)

avg_improvement = np.mean([net['improvement'] for net in optimized_networks]) * 100
print(f"\nAverage improvement: {avg_improvement:.1f}%")

## Visualize Optimization Progress

In [ ]:
# Plot fitness histories for all networks
fig, axes = plt.subplots(4, 5, figsize=(20, 16))
axes = axes.flatten()

for idx, net in enumerate(optimized_networks):
    ax = axes[idx]
    history = net['fitness_history']
    
    ax.plot(history, linewidth=2, color='steelblue', alpha=0.7)
    ax.axhline(net['initial_fitness'], color='red', linestyle='--', 
              linewidth=1, alpha=0.5, label='Initial')
    ax.axhline(net['final_fitness'], color='green', linestyle='--', 
              linewidth=1, alpha=0.5, label='Final')
    
    ax.set_xlabel('Iteration', fontsize=9)
    ax.set_ylabel('Fitness', fontsize=9)
    ax.set_title(f'Network {net["id"]+1}\nImprovement: {net["improvement"]*100:.1f}%',
                fontsize=10, fontweight='bold')
    ax.grid(alpha=0.3)
    ax.legend(fontsize=8)

plt.suptitle('Optimization Progress: All 20 Networks', 
            fontsize=16, fontweight='bold', y=0.995)
plt.tight_layout()

plt.savefig('outputs/generated/visualizations/E1_optimization_progress.svg',
           format='svg', bbox_inches='tight', dpi=300)
print("Saved: outputs/generated/visualizations/E1_optimization_progress.svg")

plt.show()

## Save Optimized Networks

In [ ]:
# Save optimized networks
with open('outputs/generated/optimized/optimized_networks_20.pkl', 'wb') as f:
    pickle.dump(optimized_networks, f)

print("✓ Saved optimized networks to: outputs/generated/optimized/optimized_networks_20.pkl")

print("\nEach optimized network includes:")
print("  - Optimized NetworkX graph")
print("  - Optimized node positions")
print("  - Fitness history (optimization trajectory)")
print("  - Initial and final fitness scores")
print("  - Improvement percentage")

print("\n" + "="*70)
print("✓ STEP 5 COMPLETE!")
print("="*70)
print(f"\nOptimized 20 networks to match {reference_city.upper()} distributions")
print(f"Average improvement: {avg_improvement:.1f}%")
print("\nNext: Rank and select best network (Step 6)")

## Summary

**Step 5 Complete!**

Optimization Results:
- Applied simulated annealing to all 20 networks
- Matched segment length distribution via KL divergence minimization
- Matched orientation distribution
- Optimized node density, average degree, segment length metrics
- 100 iterations per network with temperature cooling

**Operations applied:**
- Node position perturbations
- Edge additions (between nearby nodes)
- Edge removals (maintaining connectivity)

**Next Steps:**
1. Step 6: Rank optimized networks and select best
2. Step 7: Generate optimized buildings for selected network
3. Step 8: Export to GeoJSON/Shapefile